In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
ApparelData = pd.read_csv('apparel-trainval.csv',sep=',',index_col = False)

In [3]:
UniqueLabels=np.array([0,1,2,3,4,5,6,7,8,9])
#DataPerClass = {}
DataPerClass = []
for i in UniqueLabels:
    #DataPerClass[i] = ApparelData.loc[ApparelData['label'] == i]
    tempdf = ApparelData.loc[ApparelData['label'] == i]
    DataPerClass.append(tempdf.sample(frac = 0.8))
    print(len(DataPerClass[i]))#+str(" for ")+str(i))
train = pd.concat(DataPerClass, ignore_index= True)
train = train.sample(frac = 1)
val = ApparelData.loc[~ApparelData.index.isin(train.index)]
val = val.sample(frac = 1)

4800
4800
4800
4800
4800
4800
4800
4800
4800
4800


In [4]:
EncodedLabel = {}
EncodedLabel[0]='T-shirt/top'
EncodedLabel[1]='Trouser'
EncodedLabel[2]='Pullover'
EncodedLabel[3]='Dress'
EncodedLabel[4]='Coat'
EncodedLabel[5]='Sandal'
EncodedLabel[6]='Shirt'
EncodedLabel[7]='Sneaker'
EncodedLabel[8]='Bag'
EncodedLabel[9]='Ankle boot'

In [25]:
inputeLayerSize = 784
hiddenLayerSize = 10
outputLayerSize = 784
#numOfHiddenLayers = 1
#initialize weights and biases 
Weights ={}
Biases ={}
hiddenLayerSizes1 = [14,14,14]
numOfHiddenLayers = len(hiddenLayerSizes1)

method = 'Sigmoid'
#method = 'Relu'
#method = 'tanhx'

In [26]:
for i in range(0,len(hiddenLayerSizes1)):
    if(i==0):        
        Weights[i] = np.random.randn(hiddenLayerSizes1[i],inputeLayerSize)*np.sqrt(2/inputeLayerSize)
        Biases[i] = np.random.randn(hiddenLayerSizes1[i],1)*np.sqrt(2/inputeLayerSize)
    else:
        Weights[i] = np.random.randn(hiddenLayerSizes1[i],hiddenLayerSizes1[i-1])*np.sqrt(2/hiddenLayerSizes1[i-1])
        Biases[i] = np.random.randn(hiddenLayerSizes1[i],1)*np.sqrt(2/hiddenLayerSizes1[i-1])

Weights[numOfHiddenLayers] = np.random.randn(outputLayerSize,hiddenLayerSizes1[numOfHiddenLayers-1])*np.sqrt(2/hiddenLayerSizes1[numOfHiddenLayers-1])
Biases[numOfHiddenLayers] = np.random.randn(outputLayerSize,1)*np.sqrt(2/hiddenLayerSizes1[numOfHiddenLayers-1])

In [27]:
for i in Weights:
    Weights[i] = 0.01*Weights[i]
    Biases[i] = 0.01*Biases[i]

In [28]:
for i in Weights:
    print(Weights[i].shape)

(14, 784)
(14, 14)
(14, 14)
(784, 14)


In [29]:
def getSigmoid(x,method):
    if(method == 'Sigmoid'):
        x = np.clip(x,-500,500)
        return 1/(1+np.exp(-x))        
    elif(method == 'Relu'):
        #x = np.clip(x,-500,+500)
        return np.maximum(x, 0)
        #return (abs(x) + x) / 2         
    elif(method == 'tanhx'):
        return (2*(getSigmoid(2*x,'Sigmoid')))-1
    
def getSigmoidDash(x,method):
    if(method == 'Sigmoid'):
        tempDer3 = x.copy()
        tempDer3[tempDer3.all()>-500 and tempDer3.all()<500] = 1.0
        tempDer3[tempDer3.all()>500 and tempDer3.all()<-500] = 0.0
        #tempDer1 = np.array([1.0 if (t.any()>-500 and t.any()<500) else 0.0 for t in x])
        #tempDer1 = tempDer1.reshape(x.shape[0],x.shape[1])
        tempDer2 = getSigmoid(x,method)*(1-getSigmoid(x,method))
        return tempDer2*tempDer3
        return getSigmoid(x,method)*(1-getSigmoid(x,method))
    elif(method == 'Relu'):
        x[x<=0] = 0
        x[x>0] = 1.0        
        return x
    elif(method == 'tanhx'):
        return (1 - (getSigmoid(x,'tanhx'))**2)

In [30]:
TrainErrorInEachEpoch = []
ValErrorInEachEpoch = []
numOfIterInEachEpochs = 200
batchSize = 240
for epoch in range(0,20):
    print("Epoch is "+str(epoch))
    for j in range(0,numOfIterInEachEpochs):
        #print("Iteration is "+str(j))
        Xmatrix = train.iloc[0+(j*batchSize):batchSize+(j*batchSize),1:].values    
        Yvector = train.iloc[0+(j*batchSize):batchSize+(j*batchSize),0].values
        #print(str(0+(j*batchSize))+str(" - ")+str(batchSize+(j*batchSize)))
        #forward pass
        z = {}
        z[-1] = Xmatrix.T
        for i in range(0,numOfHiddenLayers):#np.reshape(a, (3,1))
            if(i==0):
                z[i] = (Weights[i]@getSigmoid(Xmatrix.T,method))+Biases[i]  
                print(z[i].shape)
            else:                
                #z[i-1] = np.clip( z[i-1], -500, +500 )
                #z[i]= (Weights[i]@(1./(1+np.exp(-1*z[i-1]))))+Biases[i]
                z[i]= (Weights[i]@getSigmoid(z[i-1],method))+Biases[i]
                print(z[i].shape)
        
        z[numOfHiddenLayers] = (Weights[i]@z[i-1])
        Yhat = z[numOfHiddenLayers]
        YvectorModified = z[-1]
        #YvectorModified = (Yvector[:,None]==UniqueLabels).astype(int).T
        #Yhat[Yhat==0] = 1e-10
        Cost = np.sum((np.array(Yhat-YvectorModified)**2).sum(axis=0))
        #Cost = np.sum(-1*YvectorModified*np.log(Yhat))#/batchSize
        
        
        #if(j==(numOfIterInEachEpochs-1)):
        print(Cost)
        
        #backwardprop
        
        GradWeights = {}
        GradBiases = {}

        DcostDAct = {}

        DcostDAct[numOfHiddenLayers-1] = ((Weights[numOfHiddenLayers].T)@(Yhat-YvectorModified))
        
        for temp3 in range(numOfHiddenLayers-2,-1,-1):
            DcostDAct[temp3] = (Weights[temp3+1].T)@(getSigmoidDash(z[temp3+1],method)*DcostDAct[temp3+1])                
        
        GradWeights[numOfHiddenLayers] = (Yhat-YvectorModified)@(getSigmoid(z[numOfHiddenLayers-1].T,method))
        GradBiases[numOfHiddenLayers] = (Yhat-YvectorModified).sum(axis=1)
        
        for temp1 in range(numOfHiddenLayers-1,-1,-1):
            GradWeights[temp1] = ((getSigmoidDash(z[temp1],method)*DcostDAct[temp1])@getSigmoid(z[temp1-1].T,method))
            GradBiases[temp1] = (getSigmoidDash(z[temp1],method)*DcostDAct[temp1]).sum(axis=1)
            
        #GradWeights[0] = ((getSigmoidDash(z[0],method)*DcostDAct[0])@(z[-1].T))
        #GradBiases[0] = (getSigmoidDash(z[0],method)*DcostDAct[0]).sum(axis=1) 
      
        for item in GradBiases:
            GradBiases[item] = GradBiases[item].reshape(GradBiases[item].shape[0],1)
          
        
       
        #finding gradients and updating weights and biases      
            
        for temp6 in Weights:
            Weights[temp6] =  Weights[temp6] - (0.0005*GradWeights[temp6])
        for temp7 in Biases:
            Biases[temp7] = Biases[temp7] - (0.0005*GradBiases[temp7]) #*(1/batchSize)           
        
    #trainError = getError(train,Weights,Biases,numOfHiddenLayers,'yes')
    #valError = getError(val,Weights,Biases,numOfHiddenLayers,'no')
    #TrainErrorInEachEpoch.append(trainError)
    #ValErrorInEachEpoch.append(valError)
    #print(trainError)
    #print(valError)
    #if(epoch == 14):
    #Cost = getLoss(train,Weights,Biases,numOfHiddenLayers)
    #print(Cost)
    #ValErrorInEachEpoch.append(Cost)
     #   CostAtEndOffiftyEpochs[numOfHiddenLayers]=(getLoss(train,Weights,Biases,numOfHiddenLayers))
    #if(valError > trainError):
     #   break


Epoch is 0
(14, 240)
(14, 240)
(14, 240)
(784, 240)
(784, 240)


ValueError: operands could not be broadcast together with shapes (10,240) (784,240) 

In [42]:
((Yhat)**2).sum(axis=0))

(240,)

In [43]:
np.sum((np.array(Yhat-YvectorModified)**2).sum(axis=0))

107

In [44]:
Xmatrix.shape

(240, 784)

In [1]:
import tensorflow as tf

ImportError: Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\ProgramData\Anaconda3\lib\imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "C:\ProgramData\Anaconda3\lib\imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: The specified module could not be found.


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/install/errors

for

In [3]:
conda activate tf_gpu


SyntaxError: invalid syntax (<ipython-input-3-256d845ca6de>, line 1)

In [1]:
for i in range(100000 ,100999+1):
    if i%9127==0:
        print(i)

100397


In [2]:
import itertools as it

In [4]:
import itertools as it

def is_solution(perm):
    for (i1, i2) in it.combinations(range(len(perm)), 2):
        if abs(i1 - i2) == abs(perm[i1] - perm[i2]):
            return False

    return True

for perm in it.permutations(range(12)):
    if is_solution(perm):
        print(perm)
        break

(0, 2, 4, 7, 9, 11, 5, 10, 1, 6, 8, 3)


In [10]:
def findSum(list):
    print(list)
    if(len(list)==1):
        return list[0]
    else:
        a = list[0]
        b = findSum(list[1:])
        print("calculating "+str(a)+" "+str(b))
        return a+b
a = [1,2,3,4,5]
print(findSum(a))

[1, 2, 3, 4, 5]
[2, 3, 4, 5]
[3, 4, 5]
[4, 5]
[5]
calculating 4 5
calculating 3 9
calculating 2 12
calculating 1 14
15
